In [103]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [108]:
from transformers import BertGenerationEncoder

ImportError: cannot import name 'BertGenerationEncoder' from 'transformers' (/Users/akifumiabe/.conda/envs/cat-in-the-dat-ii/lib/python3.8/site-packages/transformers/__init__.py)

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
df = pd.read_csv("../input/news_summary.csv", encoding="latin-1")
df = df[["text", "ctext"]]
df.ctext = "summarize: " + df.ctext
df.head()

,text,ctext
0,The Administration of Union Territory Daman an...,summarize: The Daman and Diu administration on...
1,Malaika Arora slammed an Instagram user who tr...,summarize: From her special numbers to TV?appe...
2,The Indira Gandhi Institute of Medical Science...,summarize: The Indira Gandhi Institute of Medi...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,summarize: Lashkar-e-Taiba's Kashmir commander...
4,Hotels in Maharashtra will train their staff t...,summarize: Hotels in Mumbai and other Indian c...


In [30]:
MAX_LEN = 512
SUMMARY_LEN = 150 
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
TRAIN_EPOCHS = 1
VAL_EPOCHS = 1 
LEARNING_RATE = 1e-4
SEED = 42
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  

In [94]:
text = str(df.text)
text = ' '.join(text.split())

ctext = str(df.ctext)
ctext = " ".join(ctext.split())

target = tokenizer.encode_plus(
    [text],
    None,
    add_special_tokens=True,
    max_length=MAX_LEN, 
    pad_to_max_length=True,
    return_token_type_ids=True,
    truncation=True,
)

source = tokenizer.encode_plus(
    [ctext],
    None,
    add_special_tokens=True,
    max_length=MAX_LEN, 
    pad_to_max_length=True,
    return_token_type_ids=True,
    truncation=True,
)

target_ids = target["input_ids"]
target_mask = target["attention_mask"]
target_token_type_ids = target["token_type_ids"]
print(target_ids)
print(target_mask)
print(target_token_type_ids)

source_ids = source["input_ids"]
source_mask = source["attention_mask"]
source_token_type_ids = source["token_type_ids"]

[101, 100, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [89]:
class CustomDataset(Dataset):
    def __init__(
        self, 
        dataframe, 
        tokenizer, 
        source_len, 
        summ_len
    ):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext
        
    def __len__(self):
        return len(self.text)
        
    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.encode_plus(
            [ctext], 
            None,
            add_special_tokens=True,
            max_length=self.source_len, 
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True,
        )
        
        target = self.tokenizer.encode_plus(
            [text],
            None,
            add_special_tokens=True,
            max_length=self.summ_len, 
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True,
        )
        
        target_ids = target["input_ids"]
        target_mask = target["attention_mask"]
        target_token_type_ids = target["token_type_ids"]

        source_ids = source["input_ids"]
        source_mask = source["attention_mask"]
        source_token_type_ids = source["token_type_ids"]
        
        return {
            "source_ids": torch.tensor(source_ids, dtype=torch.long), 
            "source_mask": torch.tensor(source_mask, dtype=torch.long),
            "source_token_type_ids": torch.tensor(source_token_type_ids, dtype=torch.long),
            "target_ids": torch.tensor(target_ids, dtype=torch.long), 
            "target_mask": torch.tensor(target_mask, dtype=torch.long),
            "target_token_type_ids": torch.tensor(target_token_type_ids, dtype=torch.long),
        }

In [90]:
train_size = 0.8
train_dataset = df.sample(frac=train_size, random_state=SEED)
val_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print(f"FULL Dataset: {df.shape}")
print(f"TRAIN Dataset: {train_dataset.shape}")
print(f"TEST Dataset: {val_dataset.shape}")

training_set = CustomDataset(
    train_dataset, 
    tokenizer, 
    MAX_LEN, 
    SUMMARY_LEN
)
    
val_set = CustomDataset(
    val_dataset, 
    tokenizer, 
    MAX_LEN, 
    SUMMARY_LEN
)

FULL Dataset: (4514, 2)
TRAIN Dataset: (3611, 2)
TEST Dataset: (903, 2)


In [91]:
train_params = {
    "batch_size": TRAIN_BATCH_SIZE,
    "shuffle": True,
    "num_workers": 0
}

val_params = {
        "batch_size": VALID_BATCH_SIZE,
        "shuffle": False,
        "num_workers": 0
}

training_loader = DataLoader(
    training_set, 
    **train_params
)
    
val_loader = DataLoader(
    val_set, 
    **val_params
)

In [92]:
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [93]:
optimizer = torch.optim.Adam(
    params=model.parameters(), 
    lr=LEARNING_RATE
)

In [101]:
for epoch in range(TRAIN_EPOCHS):
    model.train()
    for _, data in enumerate(training_loader, 0):
        source_ids = data["source_ids"].to(device, dtype=torch.long)
        source_mask = data["source_mask"].to(device, dtype=torch.long)
        source_token_type_ids = data["source_token_type_ids"].to(device, dtype=torch.long)
        target_ids = data["target_ids"].to(device, dtype=torch.long)
        target_mask = data["target_mask"].to(device, dtype=torch.long)
        target_token_type_ids = data["target_token_type_ids"].to(device, dtype=torch.long)
        
        outputs = model(
            input_ids=source_ids,
            attention_mask=source_mask,
            token_type_ids=source_token_type_ids,
            #position_ids=target_ids,
            #encoder_attention_mask=target_mask,
            #labels=target_token_type_ids,
        )
        
        loss = outputs[0]
        print(loss)
        
        #if _%500==0:
        #    print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        #optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()

tensor([[[ -6.4108,  -6.3915,  -6.3496,  ...,  -5.7395,  -5.7290,  -3.7498],
         [ -9.9286, -10.0836,  -9.8449,  ...,  -7.9650,  -8.2284,  -3.9248],
         [-11.7743, -11.4326, -11.5535,  ..., -10.4621,  -9.9159,  -6.0488],
         ...,
         [ -4.0528,  -4.1600,  -3.9803,  ...,  -4.1490,  -5.0253,   0.0801],
         [ -3.5102,  -3.2376,  -3.4060,  ...,  -2.8583,  -3.7053,   0.4102],
         [ -7.2019,  -7.1214,  -7.1733,  ...,  -6.8107,  -6.7348,  -3.1668]],

        [[ -7.0004,  -6.8637,  -6.9108,  ...,  -6.2522,  -6.3393,  -3.7306],
         [-11.1964, -11.1939, -11.1958,  ...,  -9.7810, -10.3395,  -4.5830],
         [ -8.2854,  -8.2554,  -8.4749,  ...,  -4.9942,  -5.9247,  -7.8574],
         ...,
         [ -5.2403,  -5.0797,  -5.1067,  ...,  -5.2289,  -5.7570,  -0.8729],
         [ -6.2895,  -6.0007,  -6.1122,  ...,  -5.5002,  -6.6823,  -0.3538],
         [ -6.8295,  -6.7400,  -6.8097,  ...,  -6.1811,  -6.4679,  -1.7245]]],
       grad_fn=<AddBackward0>)
tensor([[[ -6

tensor([[[ -6.9111,  -6.7878,  -6.8206,  ...,  -5.9530,  -6.0225,  -3.9293],
         [-10.0338, -10.0146, -10.1563,  ...,  -7.9081,  -9.3879,  -3.3911],
         [ -9.2153,  -9.3211,  -9.4302,  ...,  -7.8361,  -8.4466,  -6.1223],
         ...,
         [ -4.7590,  -4.4648,  -4.6052,  ...,  -4.4481,  -5.2512,   0.4812],
         [ -5.1831,  -5.1366,  -5.1438,  ...,  -5.3061,  -5.7935,  -0.2311],
         [ -6.1292,  -6.0214,  -6.0673,  ...,  -6.0712,  -6.4468,  -1.2113]],

        [[ -6.5111,  -6.4546,  -6.4520,  ...,  -5.7688,  -5.6899,  -3.7902],
         [ -9.4569,  -9.3786,  -9.5252,  ...,  -8.7242,  -9.4209,  -1.5879],
         [ -7.6269,  -7.5002,  -7.5629,  ...,  -5.7900,  -6.4830,  -5.9856],
         ...,
         [ -5.2929,  -5.2675,  -5.2987,  ...,  -5.2662,  -5.5948,  -1.1058],
         [ -6.6667,  -6.4979,  -6.6764,  ...,  -6.1929,  -6.6399,  -0.9622],
         [ -7.1871,  -7.1460,  -7.3020,  ...,  -6.5488,  -6.5475,  -2.5243]]],
       grad_fn=<AddBackward0>)
tensor([[[ -6

KeyboardInterrupt: 